In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**Importamos Librerias**

In [ ]:
!pip install ydata-profiling

In [ ]:
import pandas as pd
from ydata_profiling import ProfileReport
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
import plotly.express as px
import plotly.graph_objects as go


**Descripción del Dataset**

* amount_tsh - Total static head (amount water available to waterpoint)
* date_recorded - The date the row was entered
* funder - Who funded the well
* gps_height - Altitude of the well
* installer - Organization that installed the well
* longitude - GPS coordinate
* latitude - GPS coordinate
* wpt_name - Name of the waterpoint if there is one
* num_private -
* basin - Geographic water basin
* subvillage - Geographic location
* region - Geographic location
* region_code - Geographic location (coded)
* district_code - Geographic location (coded)
* lga - Geographic location
* ward - Geographic location
* population - Population around the well
* public_meeting - True/False
* recorded_by - Group entering this row of data
* scheme_management - Who operates the waterpoint
* scheme_name - Who operates the waterpoint
* permit - If the waterpoint is permitted
* construction_year - Year the waterpoint was constructed
* extraction_type - The kind of extraction the waterpoint uses
* extraction_type_group - The kind of extraction the waterpoint uses
* extraction_type_class - The kind of extraction the waterpoint uses
*management - How the waterpoint is managed
*management_group - How the waterpoint is managed
* payment - What the water costs
* payment_type - What the water costs
* water_quality - The quality of the water
* quality_group - The quality of the water
* quantity - The quantity of water
* quantity_group - The quantity of water
* source - The source of the water
* source_type - The source of the water
* source_class - The source of the water
* waterpoint_type - The kind of waterpoint
* waterpoint_type_group - The kind of waterpoint




**Cargamos Dataframes**

In [ ]:
train_values = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train_pump_it.csv')
train_labels = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/labels_pump_it.csv')
test_values = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test.csv')

In [ ]:
# Dimensiones (número de filas y columnas) de tres DataFrames

print("Forma del conjunto de entrenamiento:", train_values.shape)
print("Forma del conjunto de etiquetas:", train_labels.shape)
print("Forma del conjunto de prueba:", test_values.shape)

In [ ]:

for i  in train_values.columns:
    print(train_values[i].value_counts())

for i in train_labels.columns:
    print(train_labels[i].value_counts())

for i in test_values.columns:
    print(test_values[i].value_counts())

**Verificación de Valores Duplicados**

In [ ]:
print(train_values['id'].duplicated().value_counts())
print(train_labels['id'].duplicated().value_counts())
print(test_values['id'].duplicated().value_counts())


**ID´s contenidos en las tablas**

In [ ]:
pd.merge(train_values, train_labels,indicator=True )['_merge'].value_counts()

In [ ]:
print(train_values['id'].isin(train_labels['id']).value_counts())

**Merge de Target**

In [ ]:
# Unir valores y etiquetas para el conjunto de entrenamiento

df_train = pd.merge(train_values,
                    train_labels,
                    how='left', on='id')

In [ ]:
df_train.head()

**EDA  df_train**

In [ ]:
profile = ProfileReport(df_train,title="Profiling Report")

In [ ]:
profile

In [ ]:
df_train.info()

In [ ]:
df_train.describe()

In [ ]:
df_train.isnull().sum()

**Pre procesado df_train**

In [ ]:
# Identificar variables con valores nulos y su porcentaje
null_counts = df_train.isnull().sum()
null_percentages = (null_counts / len(df_train)) * 100

null_info = pd.DataFrame({
    'Null Count': null_counts,
    'Null Percentage': null_percentages
})

# Filtro para mostrar solo las columnas con valores nulos
null_info_with_nans = null_info[null_info['Null Count'] > 0].sort_values(by='Null Percentage', ascending=False)

print("Variables con valores nulos y su porcentaje en df_train:")
null_info_with_nans

In [ ]:
# Identificar columnas con alta cardinalidad
high_cardinality_cols = [col for col in df_train.columns if df_train[col].dtype == 'object' and df_train[col].nunique() > 50]

print("\nColumnas con alta cardinalidad (redundantes o irrelevantes):")
print(high_cardinality_cols)

In [ ]:
# Eliminar columnas irrelevantes
df_train = df_train.drop(columns=['num_private'], errors='ignore')
test_values = test_values.drop(columns=['num_private'], errors='ignore')

In [ ]:
# Imputación de Valores Faltantes con Unknow
for col in ['scheme_name','funder', 'installer']:
        df_train[col] = df_train[col].fillna('unknown')
        top_n = 50 # Ajusta este número según experimentación
        top_frequent_values = df_train[col].value_counts().head(top_n).index.tolist()
        df_train[col] = df_train[col].apply(lambda x: x if x in top_frequent_values else 'other_category')

In [ ]:
# Rellenar valores nulos en 'scheme_management' con la moda
mode_scheme_management = df_train['scheme_management'].mode()[0]
df_train['scheme_management'].fillna(mode_scheme_management, inplace=True)

# Rellenar valores nulos en 'public_meeting' con la moda
mode_public_meeting = df_train['public_meeting'].mode()[0]
df_train['public_meeting'].fillna(mode_public_meeting, inplace=True)

# Rellenar valores nulos en 'permit' con la moda
mode_permit = df_train['permit'].mode()[0]
df_train['permit'].fillna(mode_permit, inplace=True)

# Verificar si quedan valores nulos en las columnas tratadas
print("\nValores nulos después del tratamiento:")
print(df_train[['scheme_name','scheme_management', 'installer', 'funder', 'public_meeting', 'permit']].isnull().sum())


In [ ]:
# Manejo de Nulos y 0s en Numéricas
if 'gps_height' in df_train.columns:
    df_train['is_gps_height_zero'] = (df_train['gps_height'] == 0).astype(int)
    # Calculo de la media basado en valores non-zero
    median_gps_height = df_train[df_train['gps_height'] != 0]['gps_height'].median()
    df_train['gps_height'] = df_train['gps_height'].replace(0, median_gps_height)

if 'population' in df_train.columns:
    df_train['is_population_zero'] = (df_train['population'] == 0).astype(int)
    # Calculo de la media basado en valores non-zero
    median_population = df_train[df_train['population'] != 0]['population'].median()
    df_train['population'] = df_train['population'].replace(0, median_population)

if 'amount_tsh' in df_train.columns:
    df_train['is_amount_tsh_zero'] = (df_train['amount_tsh'] == 0).astype(int)
    # df_train['amount_tsh'] = df_train['amount_tsh'].apply(lambda x: np.log1p(x)) # Considerar log1p

In [ ]:
# Ingeniería de Características de Tiempo y Manejo de construction_year
if 'date_recorded' in df_train.columns:
    df_train['date_recorded'] = pd.to_datetime(df_train['date_recorded'])
    df_train['year_recorded'] = df_train['date_recorded'].dt.year
    df_train['month_recorded'] = df_train['date_recorded'].dt.month
    df_train['day_recorded'] = df_train['date_recorded'].dt.day
    df_train['day_of_week'] = df_train['date_recorded'].dt.dayofweek
    df_train = df_train.drop(columns=['date_recorded']) # Drop the original date_recorded column

if 'construction_year' in df_train.columns:
    df_train['is_construction_year_zero'] = (df_train['construction_year'] == 0).astype(int)
    # Calculo de la media basado en valores non-zero
    median_construction_year = df_train[df_train['construction_year'] != 0]['construction_year'].median()
    df_train['construction_year'] = df_train['construction_year'].replace(0, median_construction_year)
    if 'year_recorded' in df_train.columns:
        df_train['age_of_pump'] = df_train['year_recorded'] - df_train['construction_year']
        df_train['age_of_pump'] = df_train['age_of_pump'].apply(lambda x: max(0, x))

In [ ]:
# Eliminación de estas columnas demasiado ruidosas
df_train[['lga', 'ward', 'subvillage']] = df_train[['lga', 'ward', 'subvillage']].fillna('unknown')

In [ ]:
#  Manejo de Longitud/Latitud con 0s
if 'longitude' in df_train.columns:
    df_train['is_longitude_zero'] = (df_train['longitude'] == 0).astype(int)
    # Imputar 0s en longitud (que son errores para Tanzania) con la mediana
    # Se calcula la mediana excluyendo los valores 0
    median_longitude = df_train[df_train['longitude'] != 0]['longitude'].median()
    # Se reemplazan los valores 0 por la mediana calculada
    df_train['longitude'] = df_train['longitude'].replace(0, median_longitude)

if 'latitude' in df_train.columns:
    # Para la latitud, los valores cercanos a 0 pueden ser válidos,
    # pero valores exactamente 0 son errores para la ubicación en Tanzania
    # Se identifica si la latitud es 0 y se crea una nueva columna
    df_train['is_latitude_zero'] = (df_train['latitude'] == 0).astype(int)
    # Imputar 0s en latitud con la mediana de los valores non-zero
    # Se calcula la mediana excluyendo los valores 0
    median_latitude = df_train[df_train['latitude'] != 0]['latitude'].median()
    # Se reemplazan los valores 0 por la mediana calculada
    df_train['latitude'] = df_train['latitude'].replace(0, median_latitude)

# Verificar si quedan valores 0 en longitud y latitud después del tratamiento
print("\nNúmero de 0s en 'longitude' después del tratamiento:", (df_train['longitude'] == 0).sum())
print("Número de 0s en 'latitude' después del tratamiento:", (df_train['latitude'] == 0).sum())

In [ ]:
# Eliminación de Columnas Redundantes o Irrelevantes ---
columns_to_drop = [
    'wpt_name', # Identificadores y nombres
    'quantity',         # Redundante con 'quantity_group'
    'extraction_type', 'extraction_type_class', # 'extraction_type_group' suele ser suficiente
    'water_quality',    # Redundante con 'quality_group'
    'source_type', 'source_class', # 'source' suele ser suficiente
    'waterpoint_type',  # Redundante con 'waterpoint_type_group'
    'management_group'  # Redundante con 'management'
]
df_train = df_train.drop(columns=columns_to_drop, errors='ignore')

In [ ]:
# Muestra los valores Nan

print("Valores nulos después del preprocesado:")
print(df_train.isnull().sum()[df_train.isnull().sum() > 0])

In [ ]:
df_train.head()

**EDA test_values**

In [ ]:
profile = ProfileReport(test_values,title="Profiling Report")

In [ ]:
profile

In [ ]:
test_values.info()

In [ ]:
test_values.describe()

**Pre procesado test_values**

In [ ]:
# Identificar variables con valores nulos y su porcentaje
null_counts_test = test_values.isnull().sum()
null_percentages_test = (null_counts_test / len(test_values)) * 100

null_info_test = pd.DataFrame({
    'Null Count': null_counts_test,
    'Null Percentage': null_percentages_test
})

# Filtro para mostrar solo las columnas con valores nulos
null_info_with_nans_test = null_info_test[null_info_test['Null Count'] > 0].sort_values(by='Null Percentage', ascending=False)

print("Variables con valores nulos y su porcentaje en test_values:")
null_info_with_nans_test


In [ ]:
# Identificar columnas con alta cardinalidad
high_cardinality_cols_test = [col for col in test_values.columns if test_values[col].dtype == 'object' and test_values[col].nunique() > 50]

print("\nColumnas con alta cardinalidad (redundantes o irrelevantes) en test_values:")
print(high_cardinality_cols_test)

In [ ]:
# Imputación de Valores Faltantes con Unknow en test_values
for col in ['scheme_name','funder', 'installer']:
        test_values[col] = test_values[col].fillna('unknown')

        # Reutilizar los top_frequent_values calculados en el df_train
        top_frequent_values_train = df_train[col].value_counts().head(50).index.tolist()
        if 'other_category' not in top_frequent_values_train:
            top_frequent_values_train.append('other_category')

        test_values[col] = test_values[col].apply(lambda x: x if x in top_frequent_values_train else 'other_category')

In [ ]:
# Rellenar valores nulos en 'scheme_management' con la moda del train_values
mode_scheme_management_train = df_train['scheme_management'].mode()[0]
test_values['scheme_management'].fillna(mode_scheme_management_train, inplace=True)

# Rellenar valores nulos en 'public_meeting' con la moda del train_values
mode_public_meeting_train = df_train['public_meeting'].mode()[0]
test_values['public_meeting'].fillna(mode_public_meeting_train, inplace=True)

# Rellenar valores nulos en 'permit' con la moda del train_values
mode_permit_train = df_train['permit'].mode()[0]
test_values['permit'].fillna(mode_permit_train, inplace=True)

# Verificar si quedan valores nulos en las columnas tratadas
print("\nValores nulos después del tratamiento en test_values:")
print(test_values[['scheme_name', 'scheme_management', 'installer', 'funder', 'public_meeting', 'permit']].isnull().sum())

In [ ]:
# Manejo de Nulos y 0s en Numéricas en test_values
if 'gps_height' in test_values.columns:
    test_values['is_gps_height_zero'] = (test_values['gps_height'] == 0).astype(int)
    # Usar la mediana calculada del df_train
    test_values['gps_height'] = test_values['gps_height'].replace(0, median_gps_height)

if 'population' in test_values.columns:
    test_values['is_population_zero'] = (test_values['population'] == 0).astype(int)
    # Usar la mediana calculada del df_train
    test_values['population'] = test_values['population'].replace(0, median_population)

if 'amount_tsh' in test_values.columns:
    test_values['is_amount_tsh_zero'] = (test_values['amount_tsh'] == 0).astype(int)
    # test_values['amount_tsh'] = test_values['amount_tsh'].apply(lambda x: np.log1p(x)) # Considerar log1p si se aplicó en train

In [ ]:
# Ingeniería de Características de Tiempo y Manejo de construction_year en test_values
if 'date_recorded' in test_values.columns:
    test_values['date_recorded'] = pd.to_datetime(test_values['date_recorded'])
    test_values['year_recorded'] = test_values['date_recorded'].dt.year
    test_values['month_recorded'] = test_values['date_recorded'].dt.month
    test_values['day_recorded'] = test_values['date_recorded'].dt.day
    test_values['day_of_week'] = test_values['date_recorded'].dt.dayofweek
    test_values = test_values.drop(columns=['date_recorded']) # Drop the original date_recorded column

if 'construction_year' in test_values.columns:
    test_values['is_construction_year_zero'] = (test_values['construction_year'] == 0).astype(int)
    # Usar la mediana calculada del df_train
    test_values['construction_year'] = test_values['construction_year'].replace(0, median_construction_year)
    if 'year_recorded' in test_values.columns:
        test_values['age_of_pump'] = test_values['year_recorded'] - test_values['construction_year']
        test_values['age_of_pump'] = test_values['age_of_pump'].apply(lambda x: max(0, x))

In [ ]:
# Eliminación de estas columnas demasiado ruidosas en test_values
test_values[['lga', 'ward', 'subvillage']] = test_values[['lga', 'ward', 'subvillage']].fillna('unknown')

#  Manejo de Longitud/Latitud con 0s en test_values
if 'longitude' in test_values.columns:
    test_values['is_longitude_zero'] = (test_values['longitude'] == 0).astype(int)
    # Usar la mediana calculada del df_train
    test_values['longitude'] = test_values['longitude'].replace(0, median_longitude)

if 'latitude' in test_values.columns:
    test_values['is_latitude_zero'] = (test_values['latitude'] == 0).astype(int)
    # Usar la mediana calculada del df_train
    test_values['latitude'] = test_values['latitude'].replace(0, median_latitude)

# Verificar si quedan valores 0 en longitud y latitud después del tratamiento
print("\nNúmero de 0s en 'longitude' después del tratamiento en test_values:", (test_values['longitude'] == 0).sum())
print("Número de 0s en 'latitude' después del tratamiento en test_values:", (test_values['latitude'] == 0).sum())


In [ ]:
# Eliminación de Columnas Redundantes o Irrelevantes en test_values ---
test_values = test_values.drop(columns=columns_to_drop, errors='ignore')

print("\nValores nulos después del preprocesado en test_values:")
print(test_values.isnull().sum()[test_values.isnull().sum() > 0])

In [ ]:
test_values.head()

In [ ]:
df_train.head()

In [ ]:
# Preparación del Pipeline de Preprocesamiento y División de Datos

!pip install lightgbm
import lightgbm as lgb
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split # Import train_test_split

# Eliminar la columna 'id' ya que no es una característica para el modelo
X = df_train.drop(columns=['status_group', 'id'])
y = df_train['status_group']

# Identificar columnas categóricas y numéricas (re-evaluar si se añadieron o quitaron)
categorical_features = X.select_dtypes(include=['object', 'bool']).columns
numeric_features = X.select_dtypes(include=np.number).columns

# Crear el pipeline de preprocesamiento para características numéricas
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

# Para LightGBM, se pueden usar One-Hot Encoding o que maneje directamente categóricas.

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore')) # handle_unknown='ignore' crucial
])

# Combinar los transformadores usando ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough' # Mantener otras columnas si las hay (aunque con drop inicial no debería)
)

# Dividir los datos en conjuntos de entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Definir el modelo LightGBM
# Usamos objective='multiclass' ya que tenemos 3 clases.
# num_class=3 porque hay 3 estados posibles.
# boosting_type='gbdt' (Gradient Boosting Decision Tree) es el predeterminado y común.
model_lgbm = lgb.LGBMClassifier(objective='multiclass', num_class=3, random_state=42, n_jobs=-1)

# Crear el pipeline completo incluyendo preprocesamiento y modelo LightGBM
full_pipeline_lgbm = Pipeline(steps=[('preprocessor', preprocessor),
                                       ('classifier', model_lgbm)])


In [ ]:
# Entrenar el modelo LightGBM
# LightGBM puede manejar mejor los datos transformados si le pasamos los nombres de las columnas
# categóricas originales antes del OHE, pero con el pipeline de preprocesamiento,
# el OHE se aplica internamente. Entrenamos el pipeline completo.
full_pipeline_lgbm.fit(X_train, y_train)

# Crear el archivo de submission para LightGBM
# Hacer predicciones en el conjunto de prueba
test_ids = test_values['id']
X_test = test_values.drop(columns=['id'])

# Alinear las columnas de X_test con las de X_train antes de la transformación
X_test_aligned, X_train_aligned = X_test.align(X_train, join='inner', axis=1)

test_predictions_lgbm = full_pipeline_lgbm.predict(X_test_aligned)
submission_df_lgbm = pd.DataFrame({'id': test_ids, 'status_group': test_predictions_lgbm})

In [ ]:
# Guardar el archivo de submission
submission_df_lgbm.to_csv('submission_lgbm.csv', index=False)

print(submission_df_lgbm.head())

In [ ]:
# prompt: descargar el archivo en csv

from google.colab import files
files.download('submission_lgbm.csv')

![removed-inline-image](assets/removed_inline_image.png)